# Build Chains and Results 

In [1]:
import subprocess
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
experiment = 'EMS_DCMA_DD_prt'
tasks_types = 'tdas' # 'milestones' for chains composed of milestones without tdas
results = 'prt'# 'chains' for a list of chains only, 'prt' for chains and table
query = 'code' # 'web' if the user intends to query the results via the web shell as well
if not experiment: experiment = data_file_name.replace('.graphml', '').replace('.', '_')
from calculate import *
run_calculation_process(data_file_name, experiment, tasks_types, results, query)

Starting EC2 instance: i-0586e11281d4b02a2
EC2 instance "i-0586e11281d4b02a2" has been started
Data file uploaded to S3
Calculation finished
Compute instance stopped
Preparing results
chains list: chains.txt
chains file: chains.parquet
chains_path: EMS_DCMA_DD_prt/chains/chains.parquet
chains_list_path: EMS_DCMA_DD_prt/chains/chains.txt
downloading chains parquet file


FileNotFoundError: [Errno 2] No such file or directory: 'EMS_DCMA_DD_prt/chains/chains.parquet.6649CD9C'

# Query Results  

In [ ]:
import duckdb
con = duckdb.connect()
results_files = "'{e}/*.parquet'".format(e=experiment)
query = """SELECT * FROM {rf} """.format(rf=results_files)
query_df = con.execute(query).df()
#query_df.to_excel('query_df.xlsx')
display(query_df.head(20))

In [ ]:
len(query_df)